In [22]:
import pandas as pd
import nltk
import json

nltk.download('vader_lexicon')
nltk.download('punkt')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\zoeje\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zoeje\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
from nltk.sentiment import SentimentIntensityAnalyzer

def get_sentiment_scores(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    valence = sentiment_scores['compound']
    arousal = sentiment_scores['pos'] - sentiment_scores['neg']
    return valence, arousal

In [24]:
# TODO: this labelling might need some tweaking
def assign_emotion(valence, arousal):
    if valence >= 0.7:
        if arousal >= 0.7:
            emotion = "Excited"
        elif arousal >= 0.4:
            emotion = "Happy"
        elif arousal >= 0.1:
            emotion = "Content"
        elif arousal >= -0.1:
            emotion = "Relaxed"
        elif arousal >= -0.4:
            emotion = "Disgusted"
        elif arousal >= -0.7:
            emotion = "Sad"
        else:
            emotion = "Depressed"
    elif valence >= 0.4:
        if arousal >= 0.7:
            emotion = "Surprised"
        elif arousal >= 0.4:
            emotion = "Interested"
        elif arousal >= 0.1:
            emotion = "Curious"
        elif arousal >= -0.1:
            emotion = "Relieved"
        elif arousal >= -0.4:
            emotion = "Bored"
        elif arousal >= -0.7:
            emotion = "Disappointed"
        else:
            emotion = "Depressed"
    elif valence >= 0.1:
        if arousal >= 0.7:
            emotion = "Anxious"
        elif arousal >= 0.4:
            emotion = "Concerned"
        elif arousal >= 0.1:
            emotion = "Bored"
        elif arousal >= -0.1:
            emotion = "Calm"
        elif arousal >= -0.4:
            emotion = "Frustrated"
        elif arousal >= -0.7:
            emotion = "Sad"
        else:
            emotion = "Depressed"
    elif valence >= -0.1:
        if arousal >= 0.7:
            emotion = "Annoyed"
        elif arousal >= 0.4:
            emotion = "Irritated"
        elif arousal >= 0.1:
            emotion = "Bored"
        elif arousal >= -0.1:
            emotion = "Neutral"
        elif arousal >= -0.4:
            emotion = "Tired"
        elif arousal >= -0.7:
            emotion = "Depressed"
        else:
            emotion = "Depressed"
    elif valence >= -0.4:
        if arousal >= 0.7:
            emotion = "Fearful"
        elif arousal >= 0.4:
            emotion = "Anxious"
        elif arousal >= 0.1:
            emotion = "Worried"
        elif arousal >= -0.1:
            emotion = "Neutral"
        elif arousal >= -0.4:
            emotion = "Tired"
        elif arousal >= -0.7:
            emotion = "Depressed"
        else:
            emotion = "Depressed"
    else:
        if arousal >= 0.7:
            emotion = "Fearful"
        elif arousal >= 0.4:
            emotion = "Anxious"
        elif arousal >= 0.1:
            emotion = "Worried"
        elif arousal >= -0.1:
            emotion = "Neutral"
        elif arousal >= -0.4:
            emotion = "Tired"
        elif arousal >= -0.7:
            emotion = "Depressed"
        else:
            emotion = "Depressed"
    return emotion


In [25]:
# Example usage
text = " "
valence, arousal = get_sentiment_scores(text)
print("Valence:", valence)
print("Arousal:", arousal)

emotion = assign_emotion(valence, arousal)
print("Emotion:", emotion)


Valence: 0.0
Arousal: 0.0
Emotion: Neutral


In [26]:
# list english books
traveldf = pd.read_csv('travel_df.csv', index_col=0)
english_books = traveldf[traveldf['language'] == 'eng']
english_books.head()


for book_id in english_books['book_id']:
    try:
        with open('transcripts/no_doubles/' + str(book_id) + '.json') as f:
            book = json.load(f)
    except:
        print(book_id, "failed")
        continue

    analyzed_pages = []
    for p in book['pages']:
        page = {'text': p[0]}
        page['valence'], page['arousal'] = get_sentiment_scores(page['text'])
        page['emotion'] = assign_emotion(page['valence'], page['arousal'])

        analyzed_pages.append(page)
    book['pages'] = analyzed_pages
    with open('transcripts/emotions/' + str(book_id) + '.json', 'w') as f:
        json.dump(book, f, indent=4)
    print(book_id, "done")

856797 done
856798 done
856813 done
856815 done
856816 done
856817 done
856818 done
856820 done
856821 done
856822 done
856823 done
856824 done
856829 done
856831 done
856835 done
856838 done
856839 done
856840 done
856841 done
856842 done
856843 done
856844 done
856845 done
856846 done
856847 done
856851 done
856852 done
866611 done
866612 done
866613 done
866620 done
866621 done
867651 done
867652 done
867653 done
867654 done
867655 done


# Emotions and places

In [8]:
with open('../website/src/data/locations_per_work.json') as f:
    locations_freq=json.load(f)

In [9]:
top_10={}
for id_, cities in locations_freq.items():
    sorted_cities=sorted(cities.items(), key=lambda x: x[1], reverse=True)[:10]
    top_10[id_]= sorted_cities
print(top_10[str(856797)][0][0])

Rome


In [40]:
#locations_per_work is T in formula so now it is each cell of locations_freq
# P is nr_pages in emotions json
VS={}

for book_id in english_books['book_id']:
    try:
        with open('transcripts/emotions/' + str(book_id) + '.json') as f:
            book = json.load(f)
    except:
        print(book_id, "Emotion failed")
        continue

    try:
        with open('transcripts/cities_per_page/' + str(book_id) + '.json') as f:
            cities_per_page = json.load(f)
    except:
        print(book_id, "Cities failed")
        continue

    # we will do this for the top 10 biggest cities in each book as we will consider that it is sufficient
    VS[book_id]=[]
    for i in range(10):
        city=top_10[str(book_id)][i][0]
        VS_city=0
        P=0
        T=0   # or T=top_10[str(book_id)][i][1] ?
        for p in range(book['nr_pages']):
            #if in cities per page at page, make count otherwise pass
            if city in cities_per_page[str(p)].keys():
                P+=1
                w_p=cities_per_page[str(p)][city]
                T+=w_p
                VS_city += book['pages'][p]['valence']
                print(book['pages'][p]['valence'])

        VS_city=VS_city/P
        VS[book_id].append((city, VS_city))

        


0.9169
0.8038
0.9607
0.8271
0.743
0.6824
0.9296
0.9654
0.9117
0.9726
0.9909
0.9312
0.9838
0.9797
0.9413
0.9317
0.981
0.9837
0.9739
0.9841
0.9948
0.9951
0.9817
0.9948
0.9175
0.8514
0.9811
0.9725
0.9021
0.9878
0.9121
0.9698
0.9673
-0.5606
0.9908
0.9811
0.8804
0.9688
0.9674
-0.9887
0.9869
0.8504
0.8567
0.9786
0.9829
0.9607
-0.1298
0.9835
-0.993
0.9963
0.9946
0.9397
0.9855
0.8307
0.9806
0.9901
-0.6369
0.315
0.9904
0.913
-0.1779
0.9748
0.9753
0.9628
0.8555
0.943
0.3818
0.9072
0.9738
-0.951
0.6486
0.9471
0.9896
0.8611
0.9201
-0.7105
0.9912
0.9609
0.9565
-0.7866
0.9959
0.9939
0.9909
0.9754
0.9937
0.9957
-0.9825
0.9638
0.8872
0.6115
0.5754
0.9781
0.9897
-0.8835
0.7506
0.9505
0.8316
0.2732
0.9842
0.9376
0.743
0.6824
0.9296
0.9319
0.7845
0.9739
0.9948
0.7003
0.9908
0.9811
-0.9887
0.9165
0.9694
0.8315
0.6652
0.943
-0.951
0.9912
0.9712
0.9367
0.9777
-0.3171
0.6115
0.6774
0.8849
0.9705
0.9607
0.9837
0.9878
0.3632
0.7834
0.3818
0.9685
0.9642
0.9796
-0.951
0.9348
0.9911
0.9465
0.9471
0.8383
0.9672
0.